In [1]:
# import des lib nécessaires à la bonne exécution de spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1708004487435_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# import des lib nécessaires à la découverte et la transformation des données
from pyspark.sql.functions import isnull
from pyspark.sql.functions import year, month
from pyspark.sql.functions import when

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%bash
# script batch pour le téléchargement des données sur redfin et stockage dans aws s3
wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://store-raw-data-123-yml/city_market_tracker.tsv000.gz

--2024-02-15 14:30:05--  https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz
Resolving redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)... 52.92.203.170, 52.218.236.105, 3.5.82.147, ...
Connecting to redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)|52.92.203.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1223195819 (1.1G) [application/x-www-form-urlencoded]
Saving to: ‘STDOUT’

     0K .......... .......... .......... .......... ..........  0%  322K 61m50s
    50K .......... .......... .......... .......... ..........  0% 22.7K 7h48m
   100K .......... .......... .......... .......... ..........  0%  268M 5h12m
   150K .......... .......... .......... .......... ..........  0%  276M 3h54m
   200K .......... .......... .......... .......... ..........  0%  267M 3h7m
   250K .......... .......... .......... ........

In [3]:
# Création du spark context
spark = SparkSession.builder.appName("DataIng").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Chargement des données sur redfin dans un dataframe spark pour traitement
redfin_data = spark.read.csv("s3://store-raw-data-123-yml/city_market_tracker.tsv000.gz", header=True, inferSchema=True, sep= "\t")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Découverte du jeu de donnée

In [6]:
# Les 5 premières ligne du jeu de donnée
redfin_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+---------------+-----------+--------------+--------+----------------------+----------------+------------+--------------+----------+--------------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+----------+-------------------+--------------------+-------------+--------------------+--------------------+------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------------+--------------------+----------+--------------+--------------+------------------+--------------------+--------------------+------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-----------------------+---------------------------+----------

In [7]:
# type des features du jeu de donnée
redfin_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- period_begin: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- period_duration: integer (nullable = true)
 |-- region_type: string (nullable = true)
 |-- region_type_id: integer (nullable = true)
 |-- table_id: integer (nullable = true)
 |-- is_seasonally_adjusted: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_sale_price_mom: double (nullable = true)
 |-- median_sale_price_yoy: double (nullable = true)
 |-- median_list_price: double (nullable = true)
 |-- median_list_price_mom: double (nullable = true)
 |-- median_list_price_yoy: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- median_ppsf_mom: double (nullable = true)
 |-- median_ppsf_yoy: double (nul

In [8]:
# ensemble des colonnes 
redfin_data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['period_begin', 'period_end', 'period_duration', 'region_type', 'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region', 'city', 'state', 'state_code', 'property_type', 'property_type_id', 'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy', 'median_list_price', 'median_list_price_mom', 'median_list_price_yoy', 'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold', 'homes_sold_mom', 'homes_sold_yoy', 'pending_sales', 'pending_sales_mom', 'pending_sales_yoy', 'new_listings', 'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom', 'inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom', 'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom', 'sold_above_list_yoy', 'price_drops', 'price_drops_mom', 'price_drops_yoy', 'off_market_in_two_wee

In [9]:
# Nombre de ligne 
redfin_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5213526

## Transformation des données

In [12]:
# Pour les besoin de notre analyse, nous aurons uniquement besoin des colonnes suivantes :
# ['period_end','period_duration', 'city', 'state', 'property_type', 'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated']
news_columns = ['period_end','period_duration', 'city', 'state', 'property_type', 'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated']
redfin_data_reduce = redfin_data.select(news_columns)
redfin_data_reduce.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+------------+--------------+--------------------+-----------------+------------------+----------+---------+----------------+----------+------------------+-------------------+
|period_end|period_duration|        city|         state|       property_type|median_sale_price|       median_ppsf|homes_sold|inventory|months_of_supply|median_dom|   sold_above_list|       last_updated|
+----------+---------------+------------+--------------+--------------------+-----------------+------------------+----------+---------+----------------+----------+------------------+-------------------+
|2022-01-31|             30|    Sterling|      Virginia|     All Residential|         532000.0| 268.9473737177208|        28|       21|             0.8|        13|0.7857142857142857|2024-02-11 14:26:11|
|2013-03-31|             30|   Swansboro|North Carolina|Single Family Res...|         146250.0| 93.86333914559721|         4|       76|            19.0|       390|               0.0|2024-0

In [14]:
# nombre de valeurs null pour chaque colonne du pySpark DataFrame "redfin_data_reduce"
null_counts = [redfin_data_reduce.where(isnull(col_name)).count() for col_name in redfin_data_reduce.columns]
for i, col_name in enumerate(redfin_data_reduce.columns):
    print(f"{col_name}: {null_counts[i]} null values")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

period_end: 0 null values
period_duration: 0 null values
city: 0 null values
state: 0 null values
property_type: 0 null values
median_sale_price: 6018 null values
median_ppsf: 69155 null values
homes_sold: 5599 null values
inventory: 410377 null values
months_of_supply: 331535 null values
median_dom: 68877 null values
sold_above_list: 36137 null values
last_updated: 0 null values

In [15]:
# Suppression des lignes contenants des valeurs de features manquantes
redfin_data_reduce = redfin_data_reduce.na.drop()
redfin_data_reduce.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4715556

In [16]:
# vérifions à nouveau si notre PySpark DataFrame contient des valeurs nulls
null_counts = [redfin_data_reduce.where(isnull(col_name)).count() for col_name in redfin_data_reduce.columns]
for i, col_name in enumerate(redfin_data_reduce.columns):
    print(f"{col_name}: {null_counts[i]} null values")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

period_end: 0 null values
period_duration: 0 null values
city: 0 null values
state: 0 null values
property_type: 0 null values
median_sale_price: 0 null values
median_ppsf: 0 null values
homes_sold: 0 null values
inventory: 0 null values
months_of_supply: 0 null values
median_dom: 0 null values
sold_above_list: 0 null values
last_updated: 0 null values

In [19]:
# Création d'une nouvelle colonne "period_end_yr" contenant uniquement l'année. 
# Cette nouvelle colonne est créée à partir de la "period_end"
redfin_data_reduce = redfin_data_reduce.withColumn("period_end_yr", year(col("period_end")))

# Création d'une nouvelle colonne "period_end_month" contenant uniquement l'année. 
# Cette nouvelle colonne est créée à partir de la "period_end"
redfin_data_reduce = redfin_data_reduce.withColumn("period_end_month", month(col("period_end")))

# Vérification résulat
redfin_data_reduce[['period_end', 'period_end_yr', 'period_end_month']].show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------+----------------+
|period_end|period_end_yr|period_end_month|
+----------+-------------+----------------+
|2022-01-31|         2022|               1|
|2013-03-31|         2013|               3|
|2023-02-28|         2023|               2|
|2023-04-30|         2023|               4|
|2018-11-30|         2018|              11|
+----------+-------------+----------------+
only showing top 5 rows

In [20]:
# Supression des colonnes period_end and last_updated 
redfin_data_reduce = redfin_data_reduce.drop("period_end", "last_updated")
redfin_data_reduce.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------+--------------+--------------------+-----------------+------------------+----------+---------+----------------+----------+------------------+-------------+----------------+
|period_duration|        city|         state|       property_type|median_sale_price|       median_ppsf|homes_sold|inventory|months_of_supply|median_dom|   sold_above_list|period_end_yr|period_end_month|
+---------------+------------+--------------+--------------------+-----------------+------------------+----------+---------+----------------+----------+------------------+-------------+----------------+
|             30|    Sterling|      Virginia|     All Residential|         532000.0| 268.9473737177208|        28|       21|             0.8|        13|0.7857142857142857|         2022|               1|
|             30|   Swansboro|North Carolina|Single Family Res...|         146250.0| 93.86333914559721|         4|       76|            19.0|       390|               0.0|         2013|   

## Chargement des données au format parquet dans aws s3 Bucket

In [22]:
# Nom du bucket dans lequel le fichier transformé sera stocké
s3_bucket = "s3://redfin-transform-zone-123-yml/redfin_data.parquet"

#chargement des données
redfin_data_reduce.write.mode("overwrite").parquet(s3_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…